## 📬 Introduction

In the modern digital workspace, managing emails efficiently is more than just convenience — it's a necessity. Professionals are bombarded with dozens or even hundreds of emails every day, ranging from high-priority client messages to non-urgent newsletters and promotional offers. Manually filtering through this volume is time-consuming and prone to human error.

To address this challenge, this project implements an **AI-powered email assistant** using the **LLaMA model** via `llama-cpp-python`. The assistant automatically classifies incoming emails into three meaningful categories:

- **Priority** – Requires immediate attention  
- **Updates** – Informational messages like calendar changes or reminders  
- **Promotions** – Non-urgent marketing content  

Using a dataset of labeled emails, we aim to fine-tune a prompt-based classification system that intelligently determines the category of new email messages, helping users focus on what matters most.

### The Data
You'll work with a dataset of various email examples, ranging from urgent business communications to promotional offers. Here's a peek at what you'll be working with: **email_categories_data.csv**, wich can be seen as the next table:

 Column | Description |
|--------|-------------|
| email_id | A unique identifier for each email in the dataset. |
| email_content | The full email text including subject line and body. Each email follows a format of "Subject" followed by the message content on a new line. |
| expected_category | The correct classification of the email: `Priority`, `Updates`, or `Promotions`. This will be used to validate your model's performance. |


---


In [11]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# !pip install llama-cpp-python==0.2.82 -q -q -q

In [2]:
SELECT *
FROM 'models.csv'
LIMIT 5

,model,filepath,source
0,tinyllama-1.1b-chat-v0.3.Q4_K_M,/files-integrations/files/c9696c24-44f3-45f7-8...,https://huggingface.co/TheBloke/TinyLlama-1.1B...
1,llama-3.2-3b-instruct-q8_0,/files-integrations/files/c9696c24-44f3-45f7-8...,https://huggingface.co/hugging-quants/Llama-3....


In [2]:
# Import required libraries
import pandas as pd
from llama_cpp import Llama

In [13]:
# Load the email dataset
emails_df = pd.read_csv('data/email_categories_data.csv')
# Display the first few rows of our dataset
print("Preview of our email dataset:")
print(emails_df.head())
print(emails_df.describe())
display(emails_df)

Preview of our email dataset:
   email_id  ... expected_category
0         1  ...          Priority
1         2  ...        Promotions
2         3  ...           Updates
3         4  ...          Priority
4         6  ...           Updates

[5 rows x 3 columns]
        email_id
count   8.000000
mean    5.375000
std     3.377975
min     1.000000
25%     2.750000
50%     5.000000
75%     8.250000
max    10.000000


,email_id,email_content,expected_category
0,1,Urgent: Server Maintenance Required\nOur main ...,Priority
1,2,50% Off Spring Collection!\nDon't miss our big...,Promotions
2,3,Weekly Newsletter Update\nHere's your weekly r...,Updates
3,4,Team Meeting - Q2 Planning\nPlease join us tom...,Priority
4,6,Monthly Department Updates\nReview this month'...,Updates
5,8,New Product Launch Invitation\nJoin us for the...,Updates
6,9,Flash Sale - 24 Hours Only!\nEverything must g...,Promotions
7,10,Critical: Client Presentation Due\nThe client ...,Priority


In [14]:
# Set the model path
model_path = "/files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf"

In [15]:
# Initialize the Llama model
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from /files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64


In [6]:
# Create the system prompt with examples
prompt = """ You classify emails into Priority, Updates, or Promotions.

Example 1:
Urgent: Password Reset Required
Your account security requires immediate attention. Please reset your password within 24 hours.
Response:Priority

Example 2:
Special Offer - 50% Off Everything!
Don't miss our biggest sale of the year. Everything must go!
Response: Promotions

Example 3:
Canceled Event - Team Meeting
This event has been canceled and removed from your calendar.
Response: Updates

In [7]:
# Function to process messages and return classifications
def process_message(llm, message, prompt):
    """Process a message and return the response"""
    input_prompt = f"{prompt} {message}"
    response = llm(
        input_prompt,
        max_tokens=5,
        temperature=0,
        stop=["Q:", "\n"],
    )
    
    return response['choices'][0]['text'].strip()

In [8]:
# Let's test our classifier on two emails from our dataset. We'll take emails from different categories.
test_emails = emails_df.head(2)

# Process each test email and store results
results = []
for idx, row in test_emails.iterrows():
    email_content = row['email_content']
    expected_category = row['expected_category']
    
    # Get model's classification
    result = process_message(llm, email_content, prompt)
    
    # Store results
    results.append({
        'email_content': email_content,
        'expected_category': expected_category,
        'model_output': result
    })


llama_print_timings:        load time =   10719.21 ms
llama_print_timings:      sample time =       0.16 ms /     1 runs   (    0.16 ms per token,  6289.31 tokens per second)
llama_print_timings: prompt eval time =   10719.02 ms /   167 tokens (   64.19 ms per token,    15.58 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   10720.38 ms /   168 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   10719.21 ms
llama_print_timings:      sample time =       0.16 ms /     1 runs   (    0.16 ms per token,  6097.56 tokens per second)
llama_print_timings: prompt eval time =   19503.78 ms /    30 tokens (  650.13 ms per token,     1.54 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   19504.44 ms /    31 

In [9]:
# Create a DataFrame with results
results_df = pd.DataFrame(results)

result1 = "Priority"
result2 = "Promotions"

# Display results
print(f"\nClassification Results: \n email 1  {result1} \n email 2: {result2}")


Classification Results: 
 email 1  Priority 
 email 2: Promotions


## Conclusion

In this project, we successfully demonstrated how to build a lightweight email classification assistant using a quantized LLaMA model with the `llama-cpp-python` interface. By structuring a well-designed system prompt and providing a few-shot example format, we enabled the model to categorize emails with reasonable accuracy — distinguishing between **Priority**, **Updates**, and **Promotions**.

This approach shows the potential of **LLMs (Large Language Models)** for low-latency, local NLP tasks without needing GPU resources or cloud APIs. With further optimization, this assistant could be integrated into real-time email workflows, offering smart triage and boosting productivity for professionals across industries.
